# TensorFlow Recommenders: Quickstart
- https://www.tensorflow.org/recommenders/examples/quickstart

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.5.0.dev20201230 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
from typing import Dict, Text
import numpy as np
import tensorflow as tf 
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [4]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

Dataset movielens downloaded and prepared to /Users/wonji/tensorflow_datasets/movielens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


Dataset movielens downloaded and prepared to /Users/wonji/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [6]:
ratings = ratings.map(lambda x: {
    'movie_title': x['movie_title'],
    'user_id': x['user_id']
})

movies = movies.map(lambda x: x['movie_title'])

In [7]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)

In [9]:
user_ids_vocabulary.adapt(ratings.map(lambda x: x['user_id']))
movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [17]:
class MoviewLensModel(tfrs.Model):
    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()
        
        self.user_model = user_model
        self.movie_model = movie_model
        
        self.task = task  # retrieval task
        
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features['user_id'])
        movie_embeddings = self.movie_model(features['movie_title'])
        return self.task(user_embeddings, movie_embeddings)  # task: tfrs.tasks.Retrieval

In [18]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary, 
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary, 
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(movies.batch(128).map(movie_model)))

In [19]:
model = MoviewLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


25/25 [==============================] - 7s 241ms/step - factorized_top_k/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0025 - factorized_top_k/top_10_categorical_accuracy: 0.0065 - factorized_top_k/top_50_categorical_accuracy: 0.0515 - factorized_top_k/top_100_categorical_accuracy: 0.1130 - loss: 33093.5562 - regularization_loss: 0.0000e+00 - total_loss: 33093.5562
Epoch 2/3
25/25 [==============================] - 6s 237ms/step - factorized_top_k/top_1_categorical_accuracy: 4.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0060 - factorized_top_k/top_10_categorical_accuracy: 0.0158 - factorized_top_k/top_50_categorical_accuracy: 0.1076 - factorized_top_k/top_100_categorical_accuracy: 0.2129 - loss: 31017.9975 - regularization_loss: 0.0000e+00 - total_loss: 31017.9975
Epoch 3/3
25/25 [==============================] - 6s 229ms/step - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy

In [20]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

In [22]:
_, titles = index(np.array(['42']))
titles

<tf.Tensor: shape=(1, 10), dtype=string, numpy=
array([[b'Rent-a-Kid (1995)', b'Just Cause (1995)',
        b'Land Before Time III: The Time of the Great Giving (1995) (V)',
        b'Princess Caraboo (1994)', b'Unstrung Heroes (1995)',
        b'Only You (1994)', b'All Dogs Go to Heaven 2 (1996)',
        b'House Arrest (1996)', b'Nell (1994)',
        b'Winnie the Pooh and the Blustery Day (1968)']], dtype=object)>